# Testing
## Datenaufbereitung des Entsorgungskalenders der ERZ

Die Datenaufbereitung wird bis anhin mit SAS gemacht. Die Datenkonvertierungen sind nicht so einfach dort, aber der Prozess ist soweit automatisiert. Daher hier nur ein kurzer Test, ob es so einfacher wäre.

Datum: 08.12.2021

### Importiere die notwendigen Packages

In [3]:
pip install openpyxl geopandas altair fiona requests folium mplleaflet contextily seaborn datetime plotly

In [4]:
import pandas as pd
import pivottablejs
from pivottablejs import pivot_ui
import numpy as np
import altair as alt
import matplotlib.pyplot as plt
from datetime import datetime
import geopandas as gpd
import folium 
import openpyxl
import time

Definiere Settings. Hier das Zahlenformat von Float-Werten (z.B. *'{:,.2f}'.format* mit Komma als Tausenderzeichen), 

In [5]:
pd.options.display.float_format = '{:.0f}'.format
pd.set_option('display.width', 100)
pd.set_option('display.max_columns', 15)

### Zeitvariabeln
Bestimme den aktuellst geladenen Monat. Hier ist es der Stand vor 2 Monaten. 
Bestimme noch weitere evt. sinnvolle Zeitvariabeln

1) Zuerst die Zeitvariabeln als Strings

In [6]:
now = datetime.now() # current date and time
date_today = now.strftime("%Y-%m-%d")
year_today = now.strftime("%Y")
month_today = now.strftime("%m")
day_today = now.strftime("%d")


2) Und hier noch die Zeitvariabeln als Integers:
- `aktuellesJahr`
- `aktuellerMonat`: Der gerade jetzt aktuelle Monat
- `selectedMonat`: Der aktuellste Monat in den Daten. In der Regel zwei Monate her.

In [7]:
now = datetime.now() 
int_times = now.timetuple()

aktuellesJahr = int_times[0]
aktuellerMonat = int_times[1]

eskJahr = int_times[0] +1

print(eskJahr, aktuellesJahr, aktuellerMonat, int_times)

2022 2021 12 time.struct_time(tm_year=2021, tm_mon=12, tm_mday=30, tm_hour=14, tm_min=12, tm_sec=12, tm_wday=3, tm_yday=364, tm_isdst=-1)


#### Parsing & Formatting Strings to Dataframes
- [**Cheat Sheet**](https://strftime.org/)
- Python documentation for **strptime**: [string **parse** time, Python 3](https://docs.python.org/3/library/datetime.html#datetime.datetime.strptime)
- Python documentation for strptime/**strftime**: [string **format** time,Python 3](https://docs.python.org/3/library/datetime.html#strftime-and-strptime-behavior)



### Zeitvariabeln auf Deutsch definieren

Beim Import oder Export von Deutschen Zeitformaten. Z.B. `Mo, 13. Dezember 2021` --> `datetime.datetime.strptime(x, '%a, %d. %B %Y')` Aber nur wenn Locale auf de gesetzt ist!

In [8]:
import locale
locale.setlocale(locale.LC_TIME, "de_DE")

'de_DE'

### Importiere einen Datensatz 



- Dataset auf INTEG Datenkatalog:  https://data.integ.stadt-zuerich.ch/dataset?q=entsorgungskalender
- Dataset auf PROD Datenkatalog: https://data.stadt-zuerich.ch/dataset?q=entsorgungskalender

Pfade zu den Daten:

In [9]:
# Filepath
fp_bioabfall = r"//szh/ssz/applikationen/OGD/Daten/Quelldaten/ERZ/"+str(eskJahr)+"/input_daten/bioabfall.xlsx"
fp_cargoTram = r"//szh/ssz/applikationen/OGD/Daten/Quelldaten/ERZ/"+str(eskJahr)+"/input_daten/cargoTram.xlsx"
fp_eTram = r"//szh/ssz/applikationen/OGD/Daten/Quelldaten/ERZ/"+str(eskJahr)+"/input_daten/eTram.xlsx"
fp_karton = r"//szh/ssz/applikationen/OGD/Daten/Quelldaten/ERZ/"+str(eskJahr)+"/input_daten/karton.xlsx"
fp_kehricht = r"//szh/ssz/applikationen/OGD/Daten/Quelldaten/ERZ/"+str(eskJahr)+"/input_daten/kehricht.xlsx"
fp_papier = r"//szh/ssz/applikationen/OGD/Daten/Quelldaten/ERZ/"+str(eskJahr)+"/input_daten/papier.xlsx"
fp_sammelstellen = r"//szh/ssz/applikationen/OGD/Daten/Quelldaten/ERZ/"+str(eskJahr)+"/input_daten/sammelstellen.xlsx"
fp_sonderabfall = r"//szh/ssz/applikationen/OGD/Daten/Quelldaten/ERZ/"+str(eskJahr)+"/input_daten/sonderabfall.xlsx"
fp_textilien = r"//szh/ssz/applikationen/OGD/Daten/Quelldaten/ERZ/"+str(eskJahr)+"/input_daten/textilien.xlsx"


print(fp_textilien)

//szh/ssz/applikationen/OGD/Daten/Quelldaten/ERZ/2022/input_daten/textilien.xlsx


#### Importiere das Excelfile und konvertiere die Datumsformate.
Siehe dazu weiter oben die Settings für locale und Hinweise zu STRPTIME!


In [10]:
# Read the data

import datetime

data_bioabfall = pd.read_excel(
    fp_bioabfall, parse_dates=['Abholdatum'], date_parser=lambda x: datetime.datetime.strptime(x, '%a, %d. %B %Y'))

data_cargoTram = pd.read_excel(
    fp_cargoTram, parse_dates=['Abholdatum'], date_parser=lambda x: datetime.datetime.strptime(x, '%a, %d. %B %Y'))

data_eTram = pd.read_excel(
    fp_eTram, parse_dates=['Abholdatum'], date_parser=lambda x: datetime.datetime.strptime(x, '%a, %d. %B %Y'))

data_karton = pd.read_excel(
    fp_karton, parse_dates=['Abholdatum'], date_parser=lambda x: datetime.datetime.strptime(x, '%a, %d. %B %Y'))

data_kehricht = pd.read_excel(
    fp_kehricht, parse_dates=['Abholdatum'], date_parser=lambda x: datetime.datetime.strptime(x, '%a, %d. %B %Y'))

data_papier = pd.read_excel(
    fp_papier, parse_dates=['Abholdatum'], date_parser=lambda x: datetime.datetime.strptime(x, '%a, %d. %B %Y'))

data_sonderabfall = pd.read_excel(
    fp_sonderabfall, parse_dates=['Abholdatum'], date_parser=lambda x: datetime.datetime.strptime(x, '%a, %d. %B %Y'))

# textilsammlung ist seit 2022 nicht mehr per plz organisiert. gehört nun zu sammelstelle
#data_textilien = pd.read_excel(
    #fp_textilien, parse_dates=['Abholdatum'], date_parser=lambda x: datetime.datetime.strptime(x, '%a, %d. %B %Y'))

In [11]:
data_sammelstellen = pd.read_excel(
    fp_sammelstellen)

data_sammelstellen.head(2)

,PLZ,Sammelstelle,Oel,Glas,Metall,Textil
0,8001,Bahnhofquai vis-à-vis 5,NaN,X,X,X
1,8001,Gessnerallee vis-à-vis 36,NaN,X,X,X


In [12]:
data_sammelstellen
#data_sammelstellen.dtypes
#data_sammelstellen.shape
data_sammelstellen.isnull().sum()
#data_sammelstellen.notnull().sum()

PLZ               0
Sammelstelle      0
Oel             119
Glas              0
Metall            0
Textil           47
dtype: int64

In [13]:
data_bioabfall
#data_bioabfall.dtypes
#data_bioabfall.shape
#data_bioabfall.isnull().sum()

,PLZ,Abholdatum
0,8001,2022-01-10
1,8001,2022-01-24
2,8001,2022-02-07
3,8001,2022-02-21
4,8001,2022-02-28
...,...,...
1099,8064,2022-11-17
1100,8064,2022-11-24
1101,8064,2022-12-01
1102,8064,2022-12-15


In [14]:
data_cargoTram
#data_cargoTram.dtypes
#data_cargoTram.shape
#data_cargoTram.isnull().sum()

,PLZ,Station,Abholdatum
0,8001,Burgwies,2022-01-03
1,8001,Burgwies,2022-01-31
2,8001,Burgwies,2022-02-28
3,8001,Burgwies,2022-03-28
4,8001,Burgwies,2022-04-25
...,...,...,...
434,8064,Wartau,2022-07-15
435,8064,Wartau,2022-09-09
436,8064,Wartau,2022-10-07
437,8064,Wartau,2022-11-04


In [15]:
data_eTram
#data_eTram.dtypes
#data_eTram.shape
#data_eTram.isnull().sum()

,PLZ,Station,Abholdatum
0,8001,Burgwies,2022-01-17
1,8001,Burgwies,2022-02-14
2,8001,Burgwies,2022-03-14
3,8001,Burgwies,2022-04-11
4,8001,Burgwies,2022-05-09
...,...,...,...
454,8064,Wartau,2022-08-26
455,8064,Wartau,2022-09-23
456,8064,Wartau,2022-10-21
457,8064,Wartau,2022-11-18


In [16]:
data_karton
#data_karton.dtypes
#data_karton.shape
#data_karton.isnull().sum()

,PLZ,Abholdatum
0,8001,2022-01-05
1,8001,2022-01-19
2,8001,2022-02-02
3,8001,2022-02-16
4,8001,2022-03-02
...,...,...
602,8064,2022-10-27
603,8064,2022-11-10
604,8064,2022-11-24
605,8064,2022-12-08


In [17]:
data_kehricht 
#data_kehricht.dtypes
#data_kehricht.shape
#data_kehricht.isnull().sum()

,PLZ,Abholdatum
0,8001,2022-01-04
1,8001,2022-01-07
2,8001,2022-01-11
3,8001,2022-01-14
4,8001,2022-01-18
...,...,...
1396,8064,2022-11-29
1397,8064,2022-12-06
1398,8064,2022-12-13
1399,8064,2022-12-20


In [18]:
data_papier
#data_papier.dtypes
#data_papier.shape
#data_papier.isnull().sum()
#data_papier.notnull().sum()

,PLZ,Abholdatum
0,8001,2022-01-12
1,8001,2022-01-26
2,8001,2022-02-09
3,8001,2022-02-23
4,8001,2022-03-09
...,...,...
603,8064,2022-11-03
604,8064,2022-11-17
605,8064,2022-12-01
606,8064,2022-12-15


In [19]:
data_sonderabfall
#data_sonderabfall.dtypes
#data_sonderabfall.shape
#data_sonderabfall.isnull().sum()

,PLZ,Station,Abholdatum
0,8001,"Parkplatz am Hirschengraben 13, vor dem kanton...",2022-03-21
1,8002,Tessinerplatz,2022-09-20
2,8003,Alte evangelische Kirche / Albisriederstrasse 385,2022-03-25
3,8003,Helvetiaplatz,2022-09-19
4,8003,Tessinerplatz,2022-09-20
5,8003,Uetlibergstrasse 134,2022-04-06
6,8003,Wasserschöpfi/Küngenmatt (Bad Heuried),2022-09-22
7,8004,Helvetiaplatz,2022-09-19
8,8005,Röntgenplatz,2022-03-16
9,8006,Rigiplatz,2022-09-21


### Da das SAS-Skript mit den Sammelstellen etwas mühe hat, speichere ich es via Pandas ab.

Optionen:
DataFrame.to_csv(path_or_buf=None, sep=',', na_rep='', float_format=None, columns=None, header=True, index=True, index_label=None, mode='w', encoding=None, compression='infer', quoting=None, quotechar='"', line_terminator=None, chunksize=None, date_format=None, doublequote=True, escapechar=None, decimal='.', errors='strict', storage_options=None)

In [33]:
data_sammelstellen.to_csv("//szh/ssz/applikationen/OGD/Daten/Quelldaten/ERZ/2022/output/entsorgungskalender_sammelstellen/entsorgungskalender_sammelstellen_2022_pandas.csv", 
                          sep=',', 
                          index=False, 
                          quoting=2, 
                          quotechar='"'
                         )



## Test: importiere die mit SAS aufbereiteten CSV

In [20]:
# Filepath
fp_bioabfall_sas = r"//szh/ssz/applikationen/OGD/Daten/Quelldaten/ERZ/"+str(eskJahr)+"/output/entsorgungskalender_bioabfall/entsorgungskalender_bioabfall_"+str(eskJahr)+".csv"
fp_cargoTram_sas = r"//szh/ssz/applikationen/OGD/Daten/Quelldaten/ERZ/"+str(eskJahr)+"/output/entsorgungskalender_cargotram/entsorgungskalender_cargoTram_"+str(eskJahr)+".csv"
fp_eTram_sas = r"//szh/ssz/applikationen/OGD/Daten/Quelldaten/ERZ/"+str(eskJahr)+"/output/entsorgungskalender_etram/entsorgungskalender_eTram_"+str(eskJahr)+".csv"
fp_karton_sas = r"//szh/ssz/applikationen/OGD/Daten/Quelldaten/ERZ/"+str(eskJahr)+"/output/entsorgungskalender_karton/entsorgungskalender_karton_"+str(eskJahr)+".csv"
fp_kehricht_sas = r"//szh/ssz/applikationen/OGD/Daten/Quelldaten/ERZ/"+str(eskJahr)+"/output/entsorgungskalender_kehricht/entsorgungskalender_kehricht_"+str(eskJahr)+".csv"
fp_papier_sas = r"//szh/ssz/applikationen/OGD/Daten/Quelldaten/ERZ/"+str(eskJahr)+"/output/entsorgungskalender_papier/entsorgungskalender_papier_"+str(eskJahr)+".csv"
fp_sammelstellen_sas = r"//szh/ssz/applikationen/OGD/Daten/Quelldaten/ERZ/"+str(eskJahr)+"/output/entsorgungskalender_sammelstellen/entsorgungskalender_sammelstellen_"+str(eskJahr)+".csv"
fp_sonderabfall_sas = r"//szh/ssz/applikationen/OGD/Daten/Quelldaten/ERZ/"+str(eskJahr)+"/output/entsorgungskalender_sonderabfall/entsorgungskalender_sonderabfall_"+str(eskJahr)+".csv"

print(fp_sammelstellen_sas)

//szh/ssz/applikationen/OGD/Daten/Quelldaten/ERZ/2022/output/entsorgungskalender_sammelstellen/entsorgungskalender_sammelstellen_2022.csv


In [21]:
data_bioabfall_csv = pd.read_csv(
    fp_bioabfall_sas, 
    parse_dates=['Abholdatum']
)

data_cargoTram_csv  = pd.read_csv(
    fp_cargoTram_sas,
    parse_dates=['Abholdatum'])

data_eTram_csv  = pd.read_csv(
    fp_eTram_sas,
    parse_dates=['Abholdatum'])

data_karton_csv  = pd.read_csv(
    fp_karton_sas,
    parse_dates=['Abholdatum'])

data_kehricht_csv  = pd.read_csv(
    fp_kehricht_sas,
    parse_dates=['Abholdatum'])

data_papier_csv  = pd.read_csv(
    fp_papier_sas,
    parse_dates=['Abholdatum'])

data_sammelstellen_csv  = pd.read_csv(
    fp_sammelstellen_sas)

data_sonderabfall_csv  = pd.read_csv(
    fp_sonderabfall_sas,
    parse_dates=['Abholdatum'])


In [99]:
data_bioabfall_csv.dtypes
#data_bioabfall_csv

PLZ                    int64
Abholdatum    datetime64[ns]
dtype: object

In [100]:
data_cargoTram_csv.dtypes

PLZ                    int64
Station               object
Abholdatum    datetime64[ns]
dtype: object

In [22]:
data_sammelstellen_csv


,PLZ,Sammelstelle,Oel,Glas,Metall,Textil,abholdatum
0,8001,Bahnhofquai vis-à-vis 5,NaN,X,X,X,.
1,8001,Gessnerallee vis-à-vis 36,NaN,X,X,X,.
2,8001,Hirschengraben 13,NaN,X,X,NaN,.
3,8002,Am Schanzengraben 25 / Gartenstrasse,NaN,X,X,NaN,.
4,8002,Brunaustrasse 6 / Seestrasse,X,X,X,X,.
...,...,...,...,...,...,...,...
158,8057,Viktoriastrasse 34,NaN,X,X,X,.
159,8057,Zeppelinstrasse 43 / Spelteriniweg,NaN,X,X,NaN,.
160,8064,Grünauring vis-à-vis 25,NaN,X,X,NaN,.
161,8064,Grünauring vis-à-vis 37 / Tüffenwies,NaN,X,X,X,.
